In [1]:
# !wget https://huggingface.co/datasets/mesolitica/semisupervised-corpus/resolve/main/similarity/shuffled-train.json
# !wget https://huggingface.co/datasets/mesolitica/semisupervised-corpus/resolve/main/similarity/shuffled-test.json

In [2]:
from tqdm import tqdm
import json
import random
import torch
import numpy as np
from transformers import AutoTokenizer, AutoConfig, DebertaV2ForSequenceClassification

from bidirectional_mistral import MistralBiModel
from transformers import MistralPreTrainedModel
import torch
import numpy as np
from typing import Optional, List
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutputWithPast

In [3]:
class MistralForSequenceClassification(MistralPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.model = MistralBiModel(config)
        self.score = nn.Linear(config.hidden_size, self.num_labels, bias=False)

        # Initialize weights and apply final processing
        self.post_init()
        
    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = transformer_outputs[0][:, 0]
        logits = self.score(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + transformer_outputs[2:]
            return ((loss,) + output) if loss is not None else output
        
        return SequenceClassifierOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )

In [4]:
from transformers import AutoTokenizer, AutoConfig

In [5]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-349M-MLM-512')
# tokenizer.padding_side = 'left'
config = AutoConfig.from_pretrained('mesolitica/malaysian-mistral-349M-MLM-512')
config.problem_type = "single_label_classification"
config.label2id = {'contradiction': 0, 'entailment': 1}

In [6]:
model = MistralForSequenceClassification.from_pretrained(
    'mesolitica/malaysian-mistral-349M-MLM-512', 
    config = config,
)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mesolitica/malaysian-mistral-349M-MLM-512 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
_ = model.cuda()

In [8]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-5)

In [9]:
train_X, train_Y = [], []
with open('shuffled-train.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        train_X.append(l['src'])
        train_Y.append(l['label'])

In [10]:
test_X, test_Y = [], []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        test_X.append(l['src'])
        test_Y.append(l['label'])
        
len(test_X)

16037

In [11]:
batch_size = 4
epoch = 100

In [12]:
best_dev_acc = -np.inf
patient = 1
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        padded.pop('token_type_ids', None)
            
        loss, pred = model(**padded, return_dict = False)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 1.0)
        trainer.step()
        losses.append(float(loss))
        pbar.set_postfix(loss = float(loss))
        
        
    dev_predicted = []
    for i in range(0, len(test_X[:10000]), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
            
        padded.pop('token_type_ids', None)
        
        loss, pred = model(**padded, return_dict = False)
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('small')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

 11%|██████▎                                                     | 29031/273391 [47:56<6:20:41, 10.70it/s, loss=0.425]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 18%|██████████▍                                               | 49033/273391 [1:20:35<5:24:54, 11.51it/s, loss=0.269]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 38%|█████████████████████▊                                   | 104648/273391 [2:51:47<4:12:01, 11.16it/s, loss=0.

 16%|█████████▍                                               | 44969/273391 [1:14:01<7:31:11,  8.44it/s, loss=0.0449]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 23%|█████████████▌                                            | 63886/273391 [1:45:11<6:01:09,  9.67it/s, loss=0.529]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 30%|█████████████████▌                                        | 82593/273391 [2:15:45<7:40:17,  6.91it/s, loss=0.

  3%|██▏                                                          | 9539/273391 [15:45<7:15:09, 10.11it/s, loss=0.842]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  4%|██▌                                                         | 11622/273391 [19:14<7:00:06, 10.39it/s, loss=0.105]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 11%|██████▎                                                    | 29091/273391 [48:04<5:29:14, 12.37it/s, loss=0.0

epoch: 2, loss: 0.5091547406192717, dev_predicted: 0.7918


100%|█████████████████████████████████████████████████████████| 273391/273391 [7:30:42<00:00, 10.11it/s, loss=0.00804]


epoch: 3, loss: 0.4974466155969935, dev_predicted: 0.7991


100%|█████████████████████████████████████████████████████████| 273391/273391 [7:29:29<00:00, 10.14it/s, loss=0.00286]


epoch: 4, loss: 0.4826976018851128, dev_predicted: 0.7993


100%|█████████████████████████████████████████████████████████| 273391/273391 [7:29:09<00:00, 10.14it/s, loss=0.00721]


epoch: 5, loss: 0.4650084321067163, dev_predicted: 0.7974


In [13]:
!nvidia-smi

Fri Apr 26 19:13:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


|   0  NVIDIA GeForce RTX 3090 Ti     On  | 00000000:01:00.0 Off |                  Off |
| 30%   34C    P8              23W / 480W |  18573MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce RTX 3090 Ti     On  | 00000000:08:00.0 Off |                  Off |
| 68%   59C    P2             172W / 450W |    483MiB / 24564MiB |     93%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   G

In [14]:
real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()
    
    padded.pop('token_type_ids', None)
    o = model(**padded, use_cache = False)
    loss = o.loss
    pred = o.logits
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|█████████████████████████████████████████████████████████████████████████████| 4010/4010 [00:57<00:00, 69.83it/s]


In [15]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y[:len(real_Y)],
        digits = 5
    )
)

              precision    recall  f1-score   support

           0    0.78508   0.79197   0.78851      7643
           1    0.80905   0.80260   0.80581      8394

    accuracy                        0.79753     16037
   macro avg    0.79707   0.79728   0.79716     16037
weighted avg    0.79763   0.79753   0.79757     16037



In [ ]:
len(real_Y)

In [ ]:
len(test_Y)